## Bonusový úkol č. 2 - stahování dat z webového zdroje
Vytvořte funkci **sync()**, která získá kompletní seznam produktů (tj. včetně dalších stránek) dostupných v kategorii
https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm
a u každého produktu zjistí jeho aktuální cenu a stav skladu.
Funkce bude uchovávat získané informace a historii změn v relační databázi SQLLite3 obsahující dvě tabulky:  
* tabulku `products` a  
* tabulku `products_history`.

Struktura obou tabulek je shodná a obsahuje následující sloupce:  
* `id` TEXT - id produktu, např. OS072i1l1 (viz data-impression-id),  
* `url` TEXT - url produktu k kterému se vztahuje cena (pouze část path, viz ukázka na konci),  
* `title` TEXT - název produktu,  
* `price` DECIMAL - cena produktu s DPH k danému datu,   
* `stock_state` TEXT - stav skladu k danému datu,  
* `last_update` DATETIME - datum poslední změny hodnot záznamu v UTC  

Do tabulky `products_history` zkopírujte záznam z tabulky `products` ve chvíli, kdy se změnil nějaký sledovaný údaj (název, cena nebo stav skladu) a je potřeba aktualizovat data v tabulce `products`. Pozor, jedno `id` může mít více variant `url` s různou cenou. Při opětovném volání funkce **sync()** se prověří existence záznamu v `products`, prověří se shoda hodnot a vždy aktualizuje hodnota `last_update`, aby bylo zřejmé, ke kterému datu je informace platná.

**Předpokládaná náročnost**: 1 hodina

### Závislosti, načtení knihoven

V následující buňce deklarujte všechny závislosti

In [10]:
%pip install requests requests_cache bs4 

import requests, requests_cache, sqlite3, random, json
from bs4 import BeautifulSoup

#pro vývoj je vhodné zapnout cache (viz přednáška), pro finalní otestovaní tento řádek zakomentujte
requests_cache.install_cache('devel') 

#nadeklarujeme si novy typ sloupce DECIMAL do sqlite3, abychom měli automatický převod mezi SQLite3 a Python
from decimal import Decimal
sqlite3.register_adapter(Decimal, lambda d: str(d))
sqlite3.register_converter("DECIMAL", lambda s: Decimal(s.decode('ascii')))

You are using pip version 19.0.3, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Deklarace funkce

V následujícím boxu definujte funkci **sync(name)** s jedním parametrem (název souboru s DB), která provede zadanou operaci. 
Pro přístup k DB lze s ohledem na složitost zadání použít přímo funkcionalitu vestavěného modulu sqlite3 (viz https://docs.python.org/2/library/sqlite3.html).

**TIP**: pro získání seznamu všech produktů lze použít endpoint https://www.alza.cz/Services/EShopService.svc/Filter

Mohlo by se také hodit: https://curl.trillworks.com/

In [41]:
import json
from datetime import datetime
from decimal import Decimal
# V tomto boxu pouze implementujte funkci ale nevolejte ji (pro vývoj si vytvořte vlastní buňky).
# nezapomeňte na cookies a hlavičky, jinak se Vám může zobrazit otázka "nejste robot?"
def sync(dbfile='data.sqlite'):
    with sqlite3.connect(dbfile, detect_types=sqlite3.PARSE_DECLTYPES) as conn:
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS products
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME, PRIMARY KEY(id,url))''')
        
        c.execute('''CREATE TABLE IF NOT EXISTS products_history
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME)''')

        c.execute('''CREATE INDEX IF NOT EXISTS idx_id ON products (id)''')
        c.execute('''CREATE INDEX IF NOT EXISTS idx_idurl ON products_history (id, url)''')

        s = requests.session()      
        
        data = {
         'idCategory': 18863907, 
         'producers': '', 
         'parameters': [], 
         'idPrefix': 0, 
         'prefixType': 0, 
         'page': 1, 
         'pageTo': 999, 
         'inStock': False, 
         'newsOnly': False, 
         'commodityStatusType': None, 
         'upperDescriptionStatus': 0, 
         'branchId': -2, 
         'sort': 0, 
         'categoryType': 1, 
         'searchTerm': '', 
         'sendProducers': False, 
         'layout': 1, 
         'append': False, 
         'leasingCatId': None, 
         'yearFrom': None, 
         'yearTo': None, 
         'artistId': None, 
         'minPrice': -1, 
         'maxPrice': -1, 
         'shouldDisplayVirtooal': False, 
         'callFromParametrizationDialog': False, 
         'commodityWearType': None, 
         'scroll': 11690, 
         'hash': '#f&cst=null&cud=0&pg=1-4&prod=', 
         'counter': 3
        }  

        r = requests.post('https://www.alza.cz/Services/EShopService.svc/Filter', json=data)

        json_data = json.loads(r.text)
        json_data2 = json_data["d"]

        soup = BeautifulSoup(json_data2["Boxes"])
        soup.prettify()

        items = soup.select('div[class*="box browsingitem js-box"]')

        for item in items:
            product = item.find(class_ = "pc browsinglink")
            product_id = str(item).split('data-impression-id=', 1)[1].split('"', 2)[1]
            product_url = str(item).split('href=',1)[1].split('"',2)[1]
            product_title = str(item).split('data-impression-name=',1)[1].split('"',2)[1]
            product_price = str(item).split('data-impression-metric2=',1)[1].split('"',2)[1]
            product_price = Decimal(product_price.replace(',','.'))
            product_stockstate = str(item).split('data-impression-dimension13=',1)[1].split('"',2)[1]
            product_stockstate = product_stockstate.replace("\xa0", " ")
            product_stockstate = product_stockstate.replace("&gt;","")

            c.execute("SELECT price,stock_state,last_update FROM products WHERE id = ? AND url = ?", (product_id, product_url))
            row_new = c.fetchone()
            if row_new is None:
                c.execute("INSERT INTO products (id,url,title,price,stock_state,last_update) VALUES (?, ?, ?, ?, ?, ?)", (product_id, product_url, product_title, product_price, product_stockstate,datetime.now()))
                conn.commit()
            elif row_new[0] != product_price or row_new[1] != product_stockstate:
                c.execute("SELECT id FROM products_history WHERE id = ? AND url = ?", (product_id, product_url))
                row_old = c.fetchone()
                if row_old is None:
                    c.execute("INSERT INTO products_history (id,url,title,price,stock_state,last_update) VALUES (?, ?, ?, ?, ?, ?)", (product_id, product_url, product_title, row_new[0], row_new[1],row_new[2]))
                else:
                    c.execute("UPDATE products_history SET price = ?,stock_state = ?,last_update = ? WHERE id = ? AND url = ?", (row_new[0], row_new[1], row_new[2], product_id, product_url))

                c.execute("UPDATE products SET price = ?,stock_state = ?,last_update = ? WHERE id = ? AND url = ?", (product_price, product_stockstate,datetime.now(),product_id, product_url))

        conn.commit()        
        c.close()

### Ověření korektní funkce

Na následujícím kódu lze ověřit základní funkcionalitu. Měly byste dostat stejný výstup jako je v ukázce. Protože se však stav e-shopu může měnit, uzpůsobte si eventuelně dotaz dle potřeb. Momentálně se testuje existence produktu https://www.alza.cz/sony-alpha-7ii?dq=2286288 ev. 
https://www.alza.cz/kod/OS072i1p5.

Při ověřování korektní funkce Vaší implementace bude porovnán obsah DB vytvořený Vaší funkcí s předpokládaným obsahem DB v určitou dobu a poté znovu s několika hodinovým odstupem.

In [42]:
from contextlib import closing

sync('data.sqlite')

with sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_DECLTYPES) as conn:
    with closing(conn.cursor()) as c:
        c.execute('SELECT id, url, price FROM products WHERE id=? AND url=? AND price>20000', ('OS072i1p5','/sony-alpha-7ii?dq=2286288'))
        r = c.fetchone()
        print(r)
        assert(r != None)

        c.execute('SELECT id, url, price FROM products WHERE id=? AND price>30000', ('OF7032a',))
        r = c.fetchall()
        print(r)
        assert (len(r)>0 and '/fujifilm-x-t3?dq=5457426' in [a[1] for a in r])

print("OK")        

('OS072i1p5', '/sony-alpha-7ii?dq=2286288', Decimal('31490'))
[('OF7032a', '/fujifilm-x-t3?dq=5457426', Decimal('39990')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-levne-d5754350.htm', Decimal('36989.7')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-sleva-d5877920.htm', Decimal('33990.11'))]
OK


### Komentář
Do pole níže můžete vložit textový komentář týkající se tohoto úkolu. Např. jak dlouho Vám trvalo řešení, co bylo obtížné, co bylo se mělo více v rámci přenášky vysvětlit apod.

In [ ]:
Obtížnost větší než u prvního úkolu. Zvládnutelné, ale trvalo mi to déle než hodinu.